In [ ]:
import geopandas
import scipy.io
import pandas as pd
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

In [ ]:
krig_mesh = epro_2019.get_kriging_mesh()
# fmap = krig_mesh.get_folium_map()  # this function is not necessary, keeping it here as a reminder 

In [ ]:
# get geopandas representation of final_biomass_table
# allows us to plot transect data
df = epro_2019.final_biomass_table
gdf = geopandas.GeoDataFrame(df, 
                             geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

# reseting the index so that we can select data based on column name
gdf = gdf.reset_index()

## Plots transect points and the full mesh 

In [ ]:
# Plot transect points 
fmap = krig_mesh.plot_points(gdf, cmap_column='Transect', color='hex')

# Plot full mesh points 
fmap = krig_mesh.plot_points(krig_mesh.mesh_gdf, fmap, color='red')

# display the folium map
fmap

## Plots transect points and "no extrapolation" points 

In [ ]:
# Plot transect points 
fmap = krig_mesh.plot_points(gdf, cmap_column='Transect', color='hex')

# Plot EchoPro Matlab no extrapolation points 
mat = scipy.io.loadmat('../2019_consolidated_files/no_extrapolated_mesh_2019.mat')
no_extra_mesh = pd.DataFrame(columns=['Latitude', 'Longitude'])
no_extra_mesh['Latitude'] = mat['no_extra_mesh'][:, 0]
no_extra_mesh['Longitude'] = mat['no_extra_mesh'][:, 1]

geo_no_extra_mesh = geopandas.GeoDataFrame(no_extra_mesh, 
                                  geometry=geopandas.points_from_xy(
                                      no_extra_mesh['Longitude'], 
                                      no_extra_mesh['Latitude']))

fmap = krig_mesh.plot_points(geo_no_extra_mesh, fmap, color='red')

# display the folium map
fmap 

## Plots transect points and Polygon surrounding them

In [ ]:
# Plot transect points 
fmap = krig_mesh.plot_points(gdf, cmap_column='Transect', color='hex')

# Plot polygon of transects 

# obtain the polygon of the transects 
transect_polygon = krig_mesh.get_polygon_of_transects(gdf.set_index('Transect'), 
                                                      n_close=4, nm_to_buffer=1.25)

import folium
# Add polygon to folium map and display it
fmap.add_child(folium.GeoJson(transect_polygon))

## Plots transects, "no extrapolation" Matlab points, and Polygon based reduced mesh  

In [ ]:
# Plot transect points 
fmap = krig_mesh.plot_points(gdf, cmap_column='Transect', color='hex')

# get reduced mesh based off of a polygon
reduced_mesh_gdf = krig_mesh.reduce_grid_points(transect_polygon)

# Plot smoothed contour points 
fmap = krig_mesh.plot_points(reduced_mesh_gdf, fmap, color='blue')

# Plot matlab produced "no extrapolation" points
fmap = krig_mesh.plot_points(geo_no_extra_mesh, fmap, color='red', 
                             marker_kwargs={'fill': True, 'opacity': 0.5})
fmap

## Plots transect points and smoothed contour 

In [ ]:
# Plot transect points 
fmap = krig_mesh.plot_points(gdf, cmap_column='Transect', color='hex')

# Plot smoothed contour points 
fmap = krig_mesh.plot_points(krig_mesh.smoothed_contour_gdf, fmap, color='blue')

fmap

## Compare Matlab transformed transect points against the Python version of EchoPro

In [ ]:
# get kriging input data from Matlab version of EchoPro
# krig_input_mat = scipy.io.loadmat('../2019_consolidated_files/kriging_input_points.mat')

# x_data = krig_input_mat['xn'].flatten()
# y_data = krig_input_mat['yn'].flatten()
# field_data = krig_input_mat['var'].flatten()

In [ ]:
# # apply transformations to data using python functions
# trans_gdf = krig_mesh.apply_longitude_transformation(gdf)
# D_x = trans_df['Longitude'].max() - trans_df['Longitude'].min()
# D_y = trans_df['Latitude'].max() - trans_df['Latitude'].min()
# x, y = krig_mesh.apply_distance_transformation(trans_gdf)

In [ ]:
# %matplotlib widget
# import matplotlib.pyplot as plt

# plt.plot(x_data, y_data, 'r.')
# plt.plot(x, y, 'k*', markersize=1.25)

In [ ]:
# import numpy as np 
# print(np.allclose(x, x_data))
# print(np.allclose(y, y_data))